# XGBoost, LightGBM, OLS and NN
https://www.kaggle.com/aharless/xgboost-lightgbm-and-ols-and-nn

### parameters

In [1]:
FUDGE_FACTOR = 1.1200 # 예측에 곱함

XGB_WEIGHT = 0.6200
BASELINE_WEIGHT = 0.0100
OLS_WEIGHT = 0.0620
NN_WEIGHT = 0.0800

XGB1_WEIGHT = 0.8000 # 두 XGB 모델의 조합에서 첫 번째 가중치

BASELINE_PRED = 0.0115 # Oleg당 교육 데이터 평균을 기준


### import library 

In [2]:
### import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, StandardScaler
import lightgbm as lgb
import gc
from sklearn.linear_model import LinearRegression
import random
import datetime as dt
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, PReLU, GaussianDropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.impute import SimpleImputer

Duplicate key in file WindowsPath('C:/ProgramData/Anaconda3/envs/tens_2g/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.size:    10.0')
Duplicate key in file WindowsPath('C:/ProgramData/Anaconda3/envs/tens_2g/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 364 ('axes.titlesize:     large   # fontsize of the axes title')
Duplicate key in file WindowsPath('C:/ProgramData/Anaconda3/envs/tens_2g/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 370 ('axes.labelsize:     medium  # fontsize of the x any y labels')
Duplicate key in file WindowsPath('C:/ProgramData/Anaconda3/envs/tens_2g/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 469 ('xtick.labelsize:     medium  # fontsize of the tick labels')
Duplicate key in file WindowsPath('C:/ProgramData/Anaconda3/envs/tens_2g/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 489 ('ytick.labelsize:     medium  # fontsize of the tick labels')


### Read in raw data

In [3]:
prop = pd.read_csv('./input/properties_2016.csv')
train = pd.read_csv('./input/train_2016_v2.csv')

C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## LIGHTGBM

### Process data for lightgbm

In [4]:
for c, dtype in zip(prop.columns, prop.dtypes):
    if dtype == np.float64:
        prop[c] = prop[c].astype(np.float32)
        
df_train = train.merge(prop, how='left', on='parcelid')
df_train.fillna(df_train.median(), inplace=True)

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'],
                         axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)
    
del df_train; gc.collect()

x_train = x_train.values.astype(np.float32, copy=False)
d_train = lgb.Dataset(x_train, label=y_train)

(90275, 53) (90275,)


### Run LightGBM

In [5]:
params = {}
params['max_bin'] = 10 
params['learning_rate'] = 0.0021 
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'
params['sub_feature'] = 0.345
params['bagging_fraction'] = 0.85
params['bagging_freq'] = 40
params['num_leaves'] = 512
params['min_data'] = 500
params['min_hessian'] = 0.05
params['verbose'] = 0
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

np.random.seed(0)
random.seed(0)

print('\nFitting LightGBM Model...')
clf = lgb.train(params, d_train, 430)

del d_train; gc.collect()
del x_train; gc.collect()

print('\n Prepare for LightGBM prediction ...')
print('   Read Sample file...')
sample = pd.read_csv('./input/sample_submission.csv')
print('   ...')
sample['parcelid'] = sample['ParcelId']
print('   merge with property data...')
df_test = sample.merge(prop, on='parcelid', how='left')
print('   ...')
del sample, prop; gc.collect()
print('   ...')
x_test = df_test[train_columns]
print('   ...')
del df_test; gc.collect()
print('   Preparing x_test...')
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
print('   ...')
x_test = x_test.values.astype(np.float32, copy=False)

print('\n Start LightGBM prediction...')
p_test = clf.predict(x_test)

del x_test; gc.collect()

print('\nUnadjusted LightGBM predictions:')
print(pd.DataFrame(p_test).head())


Fitting LightGBM Model...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003074 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

## XGBoost

### Re-read properties file

In [6]:
properties = pd.read_csv('./input/properties_2016.csv')

C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Process data for xgboost

In [7]:
print('\nProcessing data for XGBoost...')
for c in properties.columns:
    properties[c] = properties[c].fillna(-1)
    if properties[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(properties[c].values))
        properties[c] = lbl.transform(list(properties[c].values))
        
train_df = train.merge(properties, how='left', on='parcelid')
x_train = train_df.drop(['parcelid', 'logerror', 'transactiondate'], axis=1)
x_test = properties.drop(['parcelid'], axis=1)

# Shape
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))

# Drop outliers
train_df = train_df[train_df.logerror > -0.4]
train_df = train_df[train_df.logerror < 0.419]
x_train = train_df.drop(['parcelid', 'logerror', 'transactiondate'], axis=1)
y_train = train_df['logerror'].values.astype(np.float32)
y_mean = np.mean(y_train)

print('After removing outliers')
print('Shape train: {}\nShape test: {}'.format(x_train.shape, x_test.shape))


Processing data for XGBoost...
Shape train: (90275, 57)
Shape test: (2985217, 57)
After removing outliers
Shape train: (88528, 57)
Shape test: (2985217, 57)


### Run Xgboost

In [8]:
print('\nSetting up data for XGBoost...')
# params
xgb_params = {
    'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'lambda': 0.8,
    'alpha': 0.4,
    'base_score': y_mean,
    'silent': 1
}
dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 250
print('num_boost_rounds='+str(num_boost_rounds))

# train model
print('\nTraining XGBoost...')
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print('\nPredicting with XGBoost...')
xgb_pred1 = model.predict(dtest)

print('\nFirst XGBoost predictions:')
print(pd.DataFrame(xgb_pred1).head())


Setting up data for XGBoost...
num_boost_rounds=250

Training XGBoost...

Predicting with XGBoost...

First XGBoost predictions:
          0
0 -0.037808
1 -0.030016
2  0.015642
3  0.038718
4 -0.010554


### Run XGB again

In [9]:
print('\nSetting up data for XGBoost...')
# params
xgb_params = {
    'eta': 0.033,
    'max_depth': 6,
    'subsample': 0.80,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

num_boost_rounds = 150
print('num_boost_rounds='+str(num_boost_rounds))

print('\nTraining XGBoost...')
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=num_boost_rounds)

print('\nPredicting with XGBoost...')
xgb_pred2 = model.predict(dtest)

print('\nFirst XGBoost predictions:')
print(pd.DataFrame(xgb_pred2).head())


Setting up data for XGBoost...
num_boost_rounds=150

Training XGBoost...

Predicting with XGBoost...

First XGBoost predictions:
          0
0 -0.075882
1 -0.022025
2  0.025625
3  0.060287
4  0.026121


### Combine Xgb results

In [10]:
xgb_pred = XGB1_WEIGHT*xgb_pred1 + (1-XGB1_WEIGHT)*xgb_pred2

print('\nCombined XGBoost predictions:')
print(pd.DataFrame(xgb_pred).head())

del train_df
del x_train
del x_test
del properties
del dtest
del dtrain
del xgb_pred1
del xgb_pred2
gc.collect()


Combined XGBoost predictions:
          0
0 -0.045423
1 -0.028418
2  0.017638
3  0.043031
4 -0.003219


73

## Neural Network

### Read in data for neural network

In [11]:
print('\n\nProcessing data for Neural Network...')
print('\nLoading train, prop and sample data...')
train = pd.read_csv('./input/train_2016_v2.csv', parse_dates=['transactiondate'])
prop = pd.read_csv('./input/properties_2016.csv')
sample = pd.read_csv('./input/sample_submission.csv')

print('Fitting Label Encoder on Properties...')
for c in prop.columns:
    prop[c] = prop[c].fillna(-1)
    if prop[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(prop[c].values))
        prop[c] = lbl.transform(list(prop[c].values))
        
print('Creating training set...')
df_train = train.merge(prop, how='left', on='parcelid')

df_train['transactiondate'] = pd.to_datetime(df_train['transactiondate'])
df_train['transactiondate_year'] = df_train['transactiondate'].dt.year
df_train['transactiondate_month'] = df_train['transactiondate'].dt.month
df_train['transactiondate_quarter'] = df_train['transactiondate'].dt.quarter
df_train['transactiondate'] = df_train['transactiondate'].dt.day

print('Filling NA/NaN values...')
df_train.fillna(-1.0)

print('Creatin x_train and y_train from df_train...')
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode', 'fireplacecnt', 'fireplaceflag'], axis=1)
y_train = df_train['logerror']

y_mean = np.mean(y_train)
print(x_train.shape, y_train.shape)
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)
    
print('Creating df_test...')
sample['parcelid'] = sample['ParcelId']

print('Merging Sample with property data...')
df_test = sample.merge(prop, on='parcelid', how='left')

df_test['transactiondate'] = pd.to_datetime('2016-11-15')
df_test['transactiondate_year'] = df_test['transactiondate'].dt.year
df_test['transactiondate_month'] = df_test['transactiondate'].dt.month
df_test['transactiondate_quarter'] = df_test['transactiondate'].dt.quarter
df_test['transactiondate_year'] = df_test['transactiondate'].dt.day
x_test = df_test[train_columns]

print('Shape of x_test:', x_test.shape)
print('Preparing x_test...')
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)



Processing data for Neural Network...

Loading train, prop and sample data...


C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fitting Label Encoder on Properties...
Creating training set...
Filling NA/NaN values...
Creatin x_train and y_train from df_train...
(90275, 56) (90275,)
Creating df_test...
Merging Sample with property data...
Shape of x_test: (2985217, 56)
Preparing x_test...


### Preprocessing

In [12]:
print('\nPreprocessing neural network data...')
imputer = SimpleImputer()
imputer.fit(x_train.iloc[:, :])
x_train = imputer.transform(x_train.iloc[:, :])
imputer.fit(x_test.iloc[:, :])
x_test = imputer.transform(x_test.iloc[:, :])

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

len_x = int(x_train.shape[1])
print('len_x is : ', len_x)


Preprocessing neural network data...
len_x is :  56


### Neural Network model

In [ ]:
print('\nSetting up neural network model...')
nn = Sequential()
nn.add(Dense(units=400, kernel_initializer='normal', input_dim=len_x))
nn.add(PReLU())
nn.add(Dropout(.4))
nn.add(Dense(units=160, kernel_initializer='normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units=64, kernel_initializer='normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.5))
nn.add(Dense(units=26, kernel_initializer='normal'))
nn.add(PReLU())
nn.add(BatchNormalization())
nn.add(Dropout(.6))
nn.add(Dense(units=1, kernel_initializer='normal'))
nn.compile(loss='mae', optimizer=Adam(lr=4e-3, decay=1e-4))

print('\nFitting neural network model...')
nn.fit(np.array(x_train), np.array(y_train), batch_size=32, epochs=70, verbose=2)

print('\nPredicting with neural network model...')
y_pred_ann = nn.predict(x_test)

print('\nPreparing results for write...')
nn_pred = y_pred_ann.flatten()
print('Type of nn_pred is:', type(nn_pred))
print('Shape of nn_pred is:', nn_pred.shape)

print('\nNeural Network predictions:')
print(pd.DataFrame(nn_pred).head())

# Clean up
del train
del prop
del sample
del x_train
del x_test
del df_train
del df_test
del y_pred_ann
gc.collect()


Setting up neural network model...

Fitting neural network model...
Epoch 1/70


## OLS

In [13]:
np.random.seed(17)
random.seed(17)
print('Processing data for OLS..')
train = pd.read_csv('./input/train_2016_v2.csv', parse_dates=['transactiondate'])
properties = pd.read_csv('./input/properties_2016.csv')
submission = pd.read_csv('./input/sample_submission.csv')
print(len(train), len(properties), len(submission))

def get_features(df):
    df['transactiondate'] = pd.to_datetime(df['transactiondate'])
    df['transactiondate_year'] = df['transactiondate'].dt.year
    df['transactiondate_month'] = df['transactiondate'].dt.month
    df['transactiondate'] = df['transactiondate'].dt.quarter
    df = df.fillna(-1.0)
    return df

def MAE(y, ypred):
    return np.sum([abs(y[i]-ypred[i]) for i in range(len(y))]) / len(y)

train = pd.merge(train, properties, how='left', on='parcelid')
y = train['logerror'].values
test = pd.merge(submission, properties, how='left', left_on='ParcelId', right_on='parcelid')
properties = [] #memory

exc = [train.columns[c] for c in range(len(train.columns)) if train.dtypes[c] == 'O'] + ['logerror','parcelid']
col = [c for c in train.columns if c not in exc]

train = get_features(train[col])
test['transactiondate'] = '2016-01-01' #should use the most common training date
test = get_features(test[col])


print("\nFitting OLS...")
reg = LinearRegression(n_jobs=-1)
reg.fit(train, y); print('fit...')
print(MAE(y, reg.predict(train)))
train = [];  y = [] #memory

test_dates = ['2016-10-01','2016-11-01','2016-12-01','2017-10-01','2017-11-01','2017-12-01']
test_columns = ['201610','201611','201612','201710','201711','201712']


Processing data for OLS..


C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (22,32,34,49,55) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


90275 2985217 2985217


C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\envs\tens_2g\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of 


Fitting OLS...
fit...
0.06836903629833616


## Combine and Save

In [14]:
print('\nCombining XGBoost, LightGBM and Baseline predictions ...')
lgb_weights = 1 - XGB_WEIGHT - BASELINE_WEIGHT - OLS_WEIGHT
lgb_weights0 = lgb_weights / (1-OLS_WEIGHT)
xgb_weights0 = XGB_WEIGHT / (1-OLS_WEIGHT)
baseline_weight0 = BASELINE_WEIGHT / (1-OLS_WEIGHT)
pred0 = 0
pred0 += xgb_weights0*xgb_pred
pred0 += baseline_weight0*BASELINE_PRED
pred0 += lgb_weights0*p_test

print('\nCombined XGB/LGB/baseline predictions:')
print(pd.DataFrame(pred0).head())

print('\nPredicting with OLS and combining with XGB/LGB/baseline predictions...')
for i in range(len(test_dates)):
    test['transactiondate'] = test_dates[i]
    pred = FUDGE_FACTOR * (OLS_WEIGHT*reg.predict(get_features(test)) + (1-OLS_WEIGHT)*pred0)
    submission[test_columns[i]] = [float(format(x, '.4f')) for x in pred]
    print('predict...', i)
    
print( "\nCombined XGB/LGB/baseline/OLS predictions:" )
print( submission.head() )


Combining XGBoost, LightGBM and Baseline predictions ...

Combined XGB/LGB/baseline predictions:
          0
0 -0.019679
1 -0.007702
2  0.015149
3  0.031406
4  0.001167

Predicting with OLS and combining with XGB/LGB/baseline predictions...
predict... 0
predict... 1
predict... 2
predict... 3
predict... 4
predict... 5

Combined XGB/LGB/baseline/OLS predictions:
   ParcelId  201610  201611  201612  201710  201711  201712
0  10754147 -0.0229 -0.0229 -0.0230 -0.0229 -0.0229 -0.0230
1  10759547 -0.0107 -0.0108 -0.0108 -0.0107 -0.0108 -0.0108
2  10843547  0.0610  0.0610  0.0610  0.0610  0.0610  0.0610
3  10859147  0.0357  0.0357  0.0357  0.0357  0.0357  0.0357
4  10879947  0.0034  0.0034  0.0034  0.0034  0.0034  0.0034


## Write the results

In [15]:

from datetime import datetime

print( "\nWriting results to disk ..." )
submission.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)

print( "\nFinished ...")




Writing results to disk ...

Finished ...
